# Расчёт метрик сервиса доставки еды

1. Подготовить SQL-запросы для расчета метрик:
- DAU (от англ. daily active users) — количество активных пользователей за день.
- Conversion Rate — коэффициент конверсии.
- Средний чек — средняя сумма покупки на пользователя.
- LTV (от англ. lifetime value) — совокупная ценность клиента за период.
- Retention Rate — коэффициент удержания пользователей.

2. Создать визуализации в DataLens:
- Для каждой метрики подготовить график для наглядного анализа

## 1. Расчет метрик

### 1. Расчёт DAU

Рассчитаем ежедневное количество активных зарегистрированных клиентов (user_id) за май и июнь 2021 года в городе Саранске. Критерием активности клиента будем считать размещение заказа. Это позволит оценить эффективность вовлечения клиентов в ключевую бизнес-цель — совершение покупки.

In [ ]:
SELECT
    log_date,
    count(distinct user_id) AS DAU
FROM analytics_events AS ae
JOIN cities AS c
    on c.city_id = ae.city_id
WHERE city_name = 'Саранск' AND (log_date between '2021-05-01' AND '2021-06-30') AND order_id is not null
GROUP BY log_date
ORDER BY log_date ASC

### 2. Расчёт Conversion Rate

Определим активность аудитории: как часто зарегистрированные пользователи переходят к размещению заказа, будет ли одинаковым этот показатель по дням или видны сезонные колебания в поведении пользователей. Рассчитаем конверсию зарегистрированных пользователей, которые посещают приложение, в активных клиентов.

In [ ]:
SELECT
    log_date, 
    round(count(distinct user_id) filter (WHERE event = 'main_page') / count(distinct user_id)::numeric, 2) AS cr
FROM analytics_events AS a
JOIN cities AS c
    on c.city_id = a.city_id
WHERE city_name = 'Саранск' AND log_date between '2021-05-01' AND '2021-06-30'
GROUP BY log_date
ORDER BY log_date ASC
LIMIT 10

### 3. Расчёт среднего чека

Рассчитаем средний чек активных клиентов в Саранске в мае и в июне.

In [ ]:
-- Рассчитываем величину комиссии с каждого заказа, отбираем заказы по дате и городу
WITH orders AS
    (SELECT *,
            revenue * commission AS commission_revenue
     FROM analytics_events
     JOIN cities ON analytics_events.city_id = cities.city_id
     WHERE revenue IS NOT NULL
         AND log_date BETWEEN '2021-05-01' AND '2021-06-30'
         AND city_name = 'Саранск')

SELECT
    DATE_TRUNC('month', log_date)::date AS "Месяц",
    count(distinct order_id) AS "Количество заказов",
    round(sum(commission_revenue)::numeric, 2) AS "Сумма комиссии",
    round(sum(commission_revenue)::numeric / count(distinct order_id), 2) AS "Средний чек"
FROM orders
GROUP BY DATE_TRUNC('month', log_date::date)
ORDER BY "Месяц" ASC

### 4. Расчёт LTV ресторанов

Определим три ресторана из Саранска с наибольшим LTV с начала мая до конца июня. Клиентами для сервиса доставки будут и рестораны, как и пользователи, которые делают заказы, поэтому считае LTV как суммарную комиссию, которая была получена от заказов в ресторане за эти два месяца.

In [ ]:
-- Рассчитываем величину комиссии с каждого заказа, отбираем заказы по дате и городу
WITH orders AS
    (SELECT analytics_events.rest_id,
            analytics_events.city_id,
            revenue * commission AS commission_revenue
     FROM analytics_events
     JOIN cities ON analytics_events.city_id = cities.city_id
     WHERE revenue IS NOT NULL
         AND log_date BETWEEN '2021-05-01' AND '2021-06-30'
         AND city_name = 'Саранск')

SELECT
    o.rest_id,
    chain AS "Название сети",
    type AS "Тип кухни",
    round(sum(commission_revenue)::numeric, 2) AS LTV
FROM orders AS o
JOIN partners AS p on o.rest_id = p.rest_id AND o.city_id = p.city_id
GROUP BY o.rest_id, chain, type
ORDER BY LTV DESC
LIMIT 3

### 5. Расчёт LTV ресторанов — самые популярные блюда

Необходимо проанализировать данные о ресторанах и их блюдах, чтобы определить вклад самых популярных блюд из двух ресторанов Саранска — «Гурманское Наслаждение» и «Гастрономический Шторм» — в общий показатель LTV.

In [ ]:
-- Рассчитываем величину комиссии с каждого заказа, отбираем заказы по дате и городу
WITH orders AS
    (SELECT analytics_events.rest_id,
            analytics_events.city_id,
            analytics_events.object_id,
            revenue * commission AS commission_revenue
     FROM analytics_events
     JOIN cities ON analytics_events.city_id = cities.city_id
     WHERE revenue IS NOT NULL
         AND log_date BETWEEN '2021-05-01' AND '2021-06-30'
         AND city_name = 'Саранск'), 

-- Рассчитываем два ресторана с наибольшим LTV 
top_ltv_restaurants AS
    (SELECT orders.rest_id,
            chain,
            type,
            ROUND(SUM(commission_revenue)::numeric, 2) AS LTV
     FROM orders
     JOIN partners ON orders.rest_id = partners.rest_id AND orders.city_id = partners.city_id
     GROUP BY 1, 2, 3
     ORDER BY LTV DESC
     LIMIT 2)

SELECT
    chain AS "Название сети",
    name AS "Название блюда",
    spicy, 
    fish,
    meat,
    ROUND(SUM(commission_revenue)::numeric, 2) AS LTV
FROM orders AS o
JOIN dishes AS d on o.object_id = d.object_id AND o.rest_id = d.rest_id
JOIN partners AS p on o.rest_id = p.rest_id AND o.city_id = p.city_id
WHERE chain in (SELECT chain FROM top_ltv_restaurants)
GROUP BY chain, name, spicy, fish, meat
ORDER BY LTV DESC
LIMIT 5

### 6. Расчёт Retention Rate

Определим показатель возвращаемости: какой процент пользователей возвращается в приложение в течение первой недели после регистрации и в какие дни. Рассчитаем показатель Retention Rate в первую неделю для всех новых пользователей в Саранске.

In [ ]:
-- Рассчитываем новых пользователей по дате первого посещения продукта
WITH new_users AS
    (SELECT DISTINCT first_date,
                     user_id
     FROM analytics_events
     JOIN cities ON analytics_events.city_id = cities.city_id
     WHERE first_date BETWEEN '2021-05-01' AND '2021-06-24'
         AND city_name = 'Саранск'),

-- Рассчитываем активных пользователей по дате события
active_users AS
    (SELECT DISTINCT log_date,
                     user_id
     FROM analytics_events
     JOIN cities ON analytics_events.city_id = cities.city_id
     WHERE log_date BETWEEN '2021-05-01' AND '2021-06-30'
         AND city_name = 'Саранск'),

-- Соединяем таблицы с новыми и активными пользователями
daily_retention AS
    (SELECT n.user_id,
    log_date,
    first_date,
    log_date - first_date AS day_since_install
    FROM new_users AS n
    JOIN active_users AS a USING (user_id)
    WHERE log_date >= first_date)

SELECT
    day_since_install,
    count(DISTINCT user_id) AS retained_users,
    round(count(DISTINCT user_id)::numeric / max(count(DISTINCT user_id)) OVER (ORDER BY day_since_install), 2) AS retention_rate
FROM daily_retention
GROUP BY day_since_install
ORDER BY day_since_install
LIMIT 8

### 7. Сравнение Retention Rate по месяцам

Разделим пользователей на две когорты по месяцу первого посещения продукта, чтобы сравнить Retention Rate этих когорт между собой.

In [ ]:
-- Рассчитываем новых пользователей по дате первого посещения продукта
WITH new_users AS
    (SELECT DISTINCT first_date,
                     user_id
     FROM analytics_events
     JOIN cities ON analytics_events.city_id = cities.city_id
     WHERE first_date BETWEEN '2021-05-01' AND '2021-06-24'
         AND city_name = 'Саранск'),

-- Рассчитываем активных пользователей по дате события
active_users AS
    (SELECT DISTINCT log_date,
                     user_id
     FROM analytics_events
     JOIN cities ON analytics_events.city_id = cities.city_id
     WHERE log_date BETWEEN '2021-05-01' AND '2021-06-30'
         AND city_name = 'Саранск'),

-- Соединяем таблицы с новыми и активными пользователями
daily_retention AS
    (SELECT new_users.user_id,
            first_date,
            log_date::date - first_date::date AS day_since_install
     FROM new_users
     JOIN active_users ON new_users.user_id = active_users.user_id
     AND log_date >= first_date)

SELECT
    DATE_TRUNC('month', first_date)::date AS "Месяц",
    day_since_install,
    COUNT(DISTINCT user_id) AS retained_users,
    ROUND(COUNT(DISTINCT user_id)::numeric / MAX(COUNT(DISTINCT user_id)) OVER (PARTITION BY CAST(DATE_TRUNC('month', first_date) AS date)), 2) AS retention_rate
FROM daily_retention
WHERE day_since_install < 8
GROUP BY "Месяц", day_since_install
ORDER BY "Месяц", day_since_install

---

## 2. Визуальзация в DataLens

**Ссылка на дашборд:** https://datalens.yandex/lup9jlu3sooq7

### 1. DAU (активные пользователи за день)

**Разброс значений:** Показатели колеблются от 17 до 85 активных пользователей в день.

**Пиковые значения:**
- Максимальный DAU (85) показывает наивысшую активность.
- Минимальный DAU (17) указывает на дни с низкой вовлеченностью.
- Средний уровень: Большинство значений сосредоточено в диапазоне 30–60, что можно считать типичной активностью.

При анализе DAU на линейном графике, можно заметить, что количество активных пользователей нестабильное - график имеет заметные колебания, как в высокой, так и низкой активности.

Стоит обратить внимание на дни, когда DAU был выше 60-70 пользователей в сутки: 
- в мае с 3 по 5, 11 и 31; 
- в июне - 3, с 7 по 11 и 15.
- Также стоит рассмотреть дни, когда DAU опускался ниже 30: 15, 16 и 22 мая; с 26 по 28 июня

В анализируемом периоде сложно проследить зависимость от дня недели или сезонные факторы, при этом, в рамках двух месяцев можно увидеть, что активность возрастает в примерно одинаковые промежутки. Нужно дополнительно проанализировать причины - возможно, в них проводились промо акции или у пользователей в эти даты поступает зарплата, поэтому они решают сделать заказ

Дополнительно стоит проанализировать периоды низкой активности. Они не имеют долгосрочной тенденции, но на графике ощущаются критично.

Также для понимания полной картины, стоит проанализировать конверсию в заказ. Возможно, присутствует корреляция между всплеском посетителей и оформлением заказа

### 2. Conversion Rate (коэффициент конверсии)

В рамках анализа линейного графика конверсии в заказ, можно заметить, что доля переходов нестабильна и подвержена колебаниям больше, чем DAU - от 18% до 43%.

На графике сложно определить паттерны поведений пользователей, при эти этом конверсия возрастает каждые 2-3 дня. Другой тенденции, как сезонность или от дня недели, не прослеживается.

При этом, стоит отдельно проанализировать пиковые значения - 43% и 41% 1, 3 и 4 мая соответственно - возможно, в эти дни проходили акции. И отдельно проанализировать дни, когда конверсия проседала - 28 мая 19% и 18 июня 18%.

### 3. Средний чек

При исследовании среднего чека по месяцам, можно заметить, что в июне средний чек был немного больше, чем в мае (135 и 147 рублей соответственно), - на фоне снижения DAU. Это может указывать на рост числа повторных заказов. 

Рост среднего чека — это положительная тенденция, если он сопровождается стабильным или растущим количеством заказов. Однако, если рост связан с повышением цен и приводит к снижению числа клиентов, это может быть тревожным сигналом. 

### 4. Retention Rate (коэффициент удержания пользователей)

Анализ общего Retention:
на день 1 возвращается только 14% пользователей, что указывает на очень низкую краткосрочную вовлеченность.
К дню 7 Retention падает до 4%, то есть 96% пользователей не возвращаются через неделю.

Начиная с дня 5, Retention держится на уровне 4%, что говорит о наличии небольшой, но стабильной группы лояльных клиентов.

Анализ Retention по месяцам:
В мае на 1 день Retention — 14% (как в общем случае).
К дню 7 Retention немного вырастает до 5%, возможно, из-за акций или улучшения сервиса в этот период.
В июне на день 1 Retention — 13% (чуть хуже, чем в мае).
К дню 7 Retention падает до 3%, что ниже майского показателя и общего уровня. Это может сигнализировать о проблемах (например, ухудшение качества доставки или конкуренция). 

При этом, кол-во пользователей и средний чек в июне больше. Это может говорить о том, что клиенты приходили единожды в рамках выгодных акций, но что-то в сервисе им не понравилось и они перестают им пользоваться в последующие дни.

Еще причины низкого RR - По информации сайта «ГородРабот.ру», за период с января по декабрь 2021 года средняя заработная плата в Саранске была 42 512 рублей, медианная — 28 503 рубля, модальная — 25 000 рублей, что заранее предупреждает о невысокой платежеспособности населения.

Стоит отметить, что быстрое снижение RR — типичная ситуация для подобных сервисов.

### 5. Топ-3 ресторанов по LTV

**LTV топ-3 ресторанов**
1. Гурманское Наслаждение (LTV: 170 679 р)
Лидер по LTV, что указывает на высокую прибыльность и лояльность клиентов.
Тип кухни: ресторан (возможно, премиальный сегмент).

2. Гастрономический Шторм (LTV: 164 908 р)
Близок к лидеру, но немного отстает.
Тип кухни: ресторан (возможно, специализируется на острых блюдах).

3. Шоколадный Рай (LTV: 61 109 р)
Значительно уступает первым двум (LTV в ~2.8 раза ниже).
Тип кухни: кондитерская — вероятно, меньший средний чек или менее частые повторные заказы.

Премиальные рестораны ("Гурманское Наслаждение" и "Гастрономический Шторм") генерируют значительно более высокий LTV по сравнению с кондитерской.
Шоколадный Рай может требовать оптимизации (например, программ лояльности или расширения ассортимента).

**LTV топ-5 блюд**
1. brokkoli zapechennaja v duhovke s jajcami i travami (41 140 р)
Ресторан: "Гастрономический Шторм".
Состав: рыбное или мясное блюдо

2. govjazhi shashliki v pesto iz kinzi (36 076 р)
Ресторан: "Гурманское Наслаждение".
Состав: рыбное или мясное блюдо.

3. medaloni iz lososja (14 946 р)
Ресторан: "Гурманское Наслаждение".
Состав: рыбное или мясное блюдо, но LTV в 2.4 раза ниже, чем у лидера.

4. myasnye ezhiki (14 337,62 ₽)
Ресторан: "Гурманское Наслаждение".
Состав: мясное блюдо.

5. teljatina s sousom iz belogo vina petrushki (13 960,66 ₽)
Ресторан: "Гастрономический Шторм".
Состав: рыбное или мясное блюдо, минимальный LTV в топ-5.


Мясные блюда доминируют в топе (5 из 5), что указывает на их высокую маржинальность или спрос, при этом немного уступают рыбные (4 из 5), что также говорит о популярности

При этом, по названию не совсем очевидно, что блюдо может состоять или из мяса, или из рыбы.


## Общие рекомендации

**Улучшение стабильности DAU:**
-	Внедрить еженедельные акции.
-	Запустить push-уведомления с персонализированными предложениями для неактивных пользователей.

**Анализ причин падений DAU:**
-	Проверить, совпадают ли низкие DAU с техническими сбоями (например, ошибки при заказе).
-	Изучить дни с минимальной активностью: возможно, требуется адаптация под будние/выходные.

**Стимулирование роста DAU:**
-	Ввести программу лояльности (например, бонусы за ежедневное использование приложения).
-	Усилить рекламу в периоды спадов (например, таргетированные кампании в соцсетях).

**Оптимизация конверсии:**
-	Упростить процесс заказа: уменьшить количество шагов, добавить автозаполнение данных.
-	Персонализация предложений: Внедрить рекомендательную систему на основе истории заказов. и отправлять персонализированные скидки для пользователей, которые добавили товары в корзину, но не завершили покупку.


**Рекомендации по Retention:**
1. По привлечению клиентов:
- День 0-1: Добавить триггеры для повторного заказа (например, скидку на следующий день).
- День 2-7: Внедрить push-уведомления или email-напоминания с персонализированными предложениями.
- Программа лояльности: Кешбэк или бесплатный десерт за повторные заказы в первую неделю.

2. По анализу оттока:
- Провести опросы среди пользователей, которые не вернулись (например, после дня 0).
- Проверить логи доставки/отмен заказов, чтобы выявить возможные сбои.

**Рекомендации по увеличению LTV**
-	Сделать упор на блюда, содержащие рыбу или мясо. При этом, меньше предлагать клиентам острые блюда (но оставить их, чтобы не отпугнуть аудиторию любителей острого)
-	Работать больше с ресторанами, так как LTV у этого сегмента больше
